In [ ]:
"""Word Count Engine
===============================
Implement a document scanning function wordCountEngine, which receives a string 
document and returns a list of all unique words in it and their number of
occurrences, sorted by the number of occurrences in a descending order. If
two or more words have the same count, they should be sorted according to
their order in the original sentence. Assume that all letters are in english
alphabet. You function should be case-insensitive, so for instance, the words
“Perfect” and “perfect” should be considered the same word.

The engine should strip out punctuation (even in the middle of a word) and
use whitespaces to separate words.

Analyze the time and space complexities of your solution. Try to optimize
for time while keeping a polynomial space complexity.

Examples:

input:  document = "Practice makes perfect. you'll only
                    get Perfect by practice. just practice!"

output: [ ["practice", "3"], ["perfect", "2"],
          ["makes", "1"], ["youll", "1"], ["only", "1"], 
          ["get", "1"], ["by", "1"], ["just", "1"] ]
"""

"""My solution :  Wrong output, un-optmized"""
from collections import defaultdict
import string

def word_count_engine(document):
    docList = document.lower().split(' ')
    wordMap = defaultdict(list)
    try:
        for wordIndex in range(0, len(docList)):
            word = docList[wordIndex]
            count = 0
            order = 0
            for letter in word:
                if letter not in string.ascii_letters:
                    word = word.replace(letter, "")
            if word in wordMap:
                count = wordMap[word][0] + 1
                order = wordMap[word][1]
            else:
                count = 1
                order = wordIndex
                
            if word == "":
                pass
            else:
                wordMap[word] = [count, order]
    except Exception as e:
        print(e)
    
    wordList = sorted(wordMap.values(), key=lambda x: x[0], reverse=True)
    output = []
    for value in wordList:
        for k, v in wordMap.items():
            if value == v:
                output.append([k,str(v[0])])
    
    return output
    
"""Correct solution :  optmized (time O(n), space O(n))
========================================================
Let document consist of of N words where M of them are unique (M ≤ N). 
The solution consists of two steps: 
1) parsing the string according to the criteria described in the problem 
and counting the number of occurrences of each word. 
2) sorting the [word, occurrence] pairs by the number of words’ occurrences 
in a descending order.

Step 1: we tokenize document into words by using whitespaces as delimiters.
For each word, we clean it from all non-alphabetic characters (digits, punctuation etc)
and convert it to lowercase to make counting case-insensitive. In this part, you should 
be leveraging whatever parsing capabilities your programming language of choice is 
providing. There is really no point of implementing functions that already exist.
As for counting, we’ll use a Map (Hash Table) to store words and their corresponding 
occurrences. A map is optimal in this case because it allows us find, store and
update operations in O(1) time complexity.

Step 2: as for the sorting part, rather than sorting the entries in the map
directly, which takes O(M⋅log(M)) - where M is number of unique words in 
document - a better solution will be to place words into an array of string
arrays indexed by the occurrence number and then iterate through the array 
in the reverse order. This is similar to a Bucket Sort. 
The proposed solution trades off a bit of space for performance, 
which may be a reasonable trade under certain circumstances.

Time Complexity: let N be the number of words in document and M 
the number of unique words in it (M ≤ N). Iterating over all words,
cleaning them and inserting them into a map takes O(N). 
The sorting step takes O(M) since notice that in the second loop, 
every word gets visited only once. The total time complexity is 
therefore O(N + M), which is O(N).

Space Complexity: wordMap takes O(M) space and the array of
strings array, counterList, takes another O(M). So, in total,
the space complexity is O(M).

Note: the reason we’re analyzing the problem complexity 
in terms of the number of words, and not number of characters 
is because the average length of an english word is ~5, 
so from a practical perspective this could be regarded as a
constant and therefore can be ignored (i.e. O(5N) = O(N))
"""
import string

def wordCountEngineOPTIMIZED(document):
    wordMap = {}
    wordList = document.lower().split(' ')
    
    for index in range(0, len(wordList)):
        currentWord = wordList[index]
        currentWordCount = 0
        
        for letter in currentWord:
            if letter not in string.ascii_letters: 
            # alternatively, you can also do ==> 
            #    if (ord(ch) >= ord('a') and ord(ch) <= ord('z')):
                currentWord = currentWord.replace(letter, "")
                            
        if currentWord in wordMap:
            currentWordCount = wordMap[currentWord] + 1
        else:
            currentWordCount = 1
                            
        if currentWord == "":
            pass
        else:
            wordMap[currentWord] = currentWordCount
                            
    counterList = [None for i in range(0, len(wordMap))]
    
    # add all words to a list indexed by the
    # corresponding occurrence number.
    for word, count in wordMap.items():
        wordCounterList = counterList[count]
        if (wordCounterList == None):
            wordCounterList = []
        wordCounterList.append(word)
        counterList[count] = wordCounterList

    # iterate through the list in reverse order (largest count to lowest count)
    # and add only non-null values to result
    result = []
    for count in range(len(counterList)-1, -1, -1):
        listOfWordsWithSameCount = counterList[count]
        if (listOfWordsWithSameCount == None or len(listOfWordsWithSameCount) == 0):
            continue
        else:
            for wordIndex in range(0, len(listOfWordsWithSameCount)):
                word = listOfWordsWithSameCount[wordIndex]
                result.append([word, str(count)])

    return result

document1 = "Practice makes perfect. you'll only get Perfect makes by practice. just practice!"
expected1 = "[['practice', '3'], ['makes', '2'], ['perfect', '2'], ['youll', '1'], ['only', '1'], ['get', '1'], ['by', '1'], ['just', '1']]" 
print("input: ", expected1)
print("\nMy solution: {}".format(word_count_engine(document1)))
print("\nCorrect solution: {}".format(wordCountEngineOPTIMIZED(document1)))
print(expected1.strip() == str(word_count_engine(document1)).strip() == str(wordCountEngineOPTIMIZED(document1)).strip())

In [ ]:
"""      
Flatten a Dictionary
input:  dict = {
            "Key1" : "1",
            "Key2" : {
                "a" : "2",
                "b" : "3",
                "c" : {
                    "d" : "3",
                    "e" : {
                        "" : "1"
                    }
                }
            }
        }

output: {
            "Key1" : "1",
            "Key2.a" : "2",
            "Key2.b" : "3",
            "Key2.c.d" : "3",
            "Key2.c.e" : "1"
        }
A recursion is natural choice for this kind of problem. 
We iterate over the keys in dict and distinguish between two cases:
If the value mapped to a key is a primitive,
we take that key and simply concatenate it to the flattened 
key we created up to this point. We then map the resultant key 
to the value in the output dictionary. If the value is a dictionary,
we do the same concatenation, but instead of mapping the result
to the value in the output dictionary,
we recurse on the value with the newly formed key.

Time Complexity: O(N), where N is the number of keys in the input dictionary. 
We visit every key in dictionary only once, hence the linear time complexity.

Space Complexity: O(N) since the output dictionary is asymptotically as big as 
the input dictionary. We also store recursive calls in the execution 
stack which in the worst case scenario could be O(N), as well. 
The total is still O(N).
"""

def flattenDictionary(dictionary):
    flatDictionary = {}
    flattenDictionaryHelper("", dictionary, flatDictionary)
    
    return flatDictionary

def removeLastDot(key):
    if key.endswith("."):
        return key[:-1]
    else:
        return key

def flattenDictionaryHelper(initialKey, dictionary, flatDictionary):
    for key in dictionary.keys():
        value = dictionary[key]

        if (type(value) != dict): # the value is of a primitive type
            if ((initialKey == None) | (initialKey == "")):
                flatDictionary[key] = value
            else:
                newKey = removeLastDot(initialKey + "." + key)
                flatDictionary[newKey] = value
        else:
            if ((initialKey == None) | (initialKey == "")):
                flattenDictionaryHelper(key, value, flatDictionary)
            else:
                newKey = removeLastDot(initialKey + "." + key)
                flattenDictionaryHelper(newKey, value, flatDictionary)

d = { "Key1" : "1",
      "Key2" : {
        "a" : "2",
        "b" : 3,
        "c" : {
            "d" : "4",
            "e" : {
                "" : 5,
                'k': 6
            }
        }
    }
}
print(flattenDictionary(d))

In [ ]:
"""
An H-tree is a geometric shape that consists of a repeating pattern
resembles the letter “H”.

It can be constructed by starting with a line segment 
of arbitrary length, drawing two segments of the same length
at right angles to the first through its endpoints, and continuing 
in the same vein, reducing (dividing) the length of the line segments
drawn at each stage by √2.

Write a function drawHTree that constructs an H-tree, 
given its center (x and y coordinates), a starting length,
and depth. Assume that the starting line is parallel to the X-axis.

Use the function drawLine provided to implement your algorithm.
In a production code, a drawLine function would render a real line
between two points. However, this is not a real production environment,
so to make things easier, implement drawLine such that it simply prints
its arguments (the print format is left to your discretion).

Analyze the time and space complexity of your algorithm.
In your analysis, assume that drawLine's time and space complexities
are constant, i.e. O(1).

We will start from the center point. Compute the coordinates of the 4 
tips of the H. Then we shall draw the 3 line segments of the H, i.e. 
left and right vertical of the H, and the connection of the two vertical 
segments. We will update the length and recursively draw 4 half-size 
H-trees of order one less than the current depth.

Time Complexity: every call of drawHTree invokes 
9 expressions whose time complexity is O(1) and 4 calls of drawHTree
until depth(denoted here as D) reaches to 0. 
Therefore: T(D) = 9 + 4 * T(D-1), where T is the time complexity
function and D is the depth of the H-Tree. 
Now, if we expand T(D-1) recursively all the way to T(0),
it’ll be easy to see that T(D) = O(4^D).

Space Complexity:  recursive calls add overhead since we store them 
in the execution stack. The space occupied in the stack will be then O(D),
in the worst case scenario. 
The stack space occupied will be no more than O(D) at any given
point since a sibling drawHTree will not be called before the current
one being executed returns (i.e. finishes its execution).
"""
import math 
import turtle

def drawLine(x1, y1, x2, y2):
    # draws line, assume implementation available
    print("({x1},{y1}) , ({x2},{y2})".format(x1=x1, y1=y1, x2=x2, y2=y2))
    point1 = (x1, y1)
    point2 = (x2, y2)
    turtle.penup()
    turtle.goto(point1)
    turtle.pendown()
    turtle.goto(point2)
    #turtle.hideturtle()
    turtle.exitonclick()

def drawHTree(x, y, length, depth):
    # recursion base case
    if (depth == 0):
        return

    x1 = x - length/2
    x2 = x + length/2
    y1 = y - length/2
    y2 = y + length/2

    # draw the 3 line segments of the H-Tree
    drawLine(x1, y1, x1, y2)    # left segment
    drawLine(x2, y1, x2, y2)    # right segment
    drawLine(x1, y,  x2,  y)    # connecting segment

    # at each stage, the length of segments decreases by a factor of √2
    newLength = length/math.sqrt(2)

    # decrement depth by 1 and draw an H-tree
    # at each of the tips of the current ‘H’
    drawHTree(x1, y1, newLength, depth-1)     # lower left  H-tree
    drawHTree(x1, y2, newLength, depth-1)     # upper left  H-tree
    drawHTree(x2, y1, newLength, depth-1)     # lower right H-tree
    drawHTree(x2, y2, newLength, depth-1)     # upper right H-tree

drawHTree(0.0, 0.0, 140, 3)

In [ ]:
"""
Pairs with Specific Difference
Given an array arr of distinct integers and a nonnegative integer k, 
write a function findPairsWithGivenDifference that returns an array of
all pairs [x,y] in arr, such that x - y = k. If no such pairs exist, 
return an empty array.

Note: the order of the pairs in the output array should maintain the
order of the y element in the original array.
Examples:
    input:  arr = [0, -1, -2, 2, 1], k = 1
    output: [[1, 0], [0, -1], [-1, -2], [2, 1]]

    input:  arr = [1, 7, 5, 3, 32, 17, 12], k = 17
    output: []

Constraints:
    [time limit] 5000ms
    [input] array.integer arr
    0 ≤ arr.length ≤ 100
    [input]integer k
    k ≥ 0
    [output] array.array.integer

Pairs with Specific Difference
A naive approach is is to run two loops. 
The outer loop picks the first element (smaller element) 
and the inner loop looks up for the element picked 
by the outer loop plus k. 
While this solution is done in O(1) space complexity, 
its time complexity is O(N^2), which isn’t asymptotically optimal.

We can use a hash map to improve the time complexity to O(N⋅log(N))
for the worst case and O(N) for the average case. 
We rely on the fact that if x - y = k then x - k = y.

The first step is to traverse the array, and for each element arr[i], 
we add a key-value pair of (arr[i] - k, arr[i]) to a hash map. 
Once the map is populated, we traverse the array again, 
and check for each element if a match exists in the map.

Both the first and second steps take O(N⋅log(N)) for the worst
case and O(N) for the average case. So the overall time complexity
is O(N) for the average case.
    
"""
import itertools

# My Solution: Time: O(n^2), Space:  O(n)
def find_pairs_with_given_difference(arr, k):
    if (len(arr) == 0):
        return []
    
    if k < 0:
        return
    
    result_arr = []
    for elem in arr:
        for (x,y) in itertools.product(arr, [elem]):
            if x - y == k:
                #print("({x},{y}) = {k}: TRUE".format(x=x, y=y, k=k))
                result_arr.append([x, y])
            else:
                pass
    return result_arr

# Correct Solution: Time: O(N) , Space O(n)
def findPairsWithGivenDifference(arr, k):
    # since we don't allow duplicates, no pair can satisfy x - 0 = y
    if k == 0:
        return []
        
    dictionary = {}
    answer = []
    """
    if x - y = k, then
    x - k = y
    """
    for x in arr:
        y = x - k
        dictionary[y] = x
    
    for y in arr:
        if y in dictionary:
            x = dictionary[y]
            answer.append([x, y]) 
            
    return answer

arr = [0, -1, -2, 2, 1]
k = 1
arr2 = [1, 7, 5, 3, 32, 17, 12]
k2 = 17

print("My Brute-force solution: O(n^2)")
print("result1:", find_pairs_with_given_difference(arr, k))
print("result2: ", find_pairs_with_given_difference(arr2, k2))

print("\nFaster solution: O(2n), i.e., O(n)")
print("result1:", findPairsWithGivenDifference(arr, k))
print("result2: ", findPairsWithGivenDifference(arr2, k2))

In [ ]:
"""
====================== Award Budget Cuts ============================
The awards committee of your alma mater (i.e. your college/university) 
asked for your assistance with a budget allocation problem they’re facing. 
Originally, the committee planned to give N research grants this year.
However, due to spending cutbacks, the budget was reduced to newBudget 
dollars and now they need to reallocate the grants. The committee made a 
decision that they’d like to impact as few grant recipients as possible by
applying a maximum cap on all grants. Every grant initially planned to be 
higher than cap will now be exactly cap dollars. Grants less or equal to cap, 
obviously, won’t be impacted.

Given an array grantsArray of the original grants and the reduced budget 
newBudget, write a function findGrantsCap that finds in the most efficient
manner a cap such that the least number of recipients is impacted and that
the new budget constraint is met (i.e. sum of the N reallocated grants 
equals to newBudget).

Analyze the time and space complexities of your solution.

Example:
input: 

output: 47 # and given this cap the new grants array would be
           # [2, 47, 47, 47, 47]. Notice that the sum of the
           # new grants is indeed 190
           
Constraints:
    [time limit] 5000ms
    [input] array.double grantsArray
    0 ≤ grantsArray.length ≤ 20
    0 ≤ grantsArray[i]
    [input] double newBudget
    [output] double
"""
def find_grants_cap(grantsArray, newBudget):
    # sort the array in a descending order.
    arr = sorted(grantsArray, reverse=True)
    print(arr)
    
    # pad the array with a zero at the end to
    # cover the case where 0 <= cap <= grantsArray[i]
    arr.append(0)
    n = len(grantsArray)
    oldBudget = 0
    for i in arr:
        oldBudget += i
    
    
    # calculate the total amount we need to
    # cut back to meet the reduced budget
    surplus = oldBudget - newBudget

    # if there is nothing to cut, simply return
    # the highest grant as the cap. Recall that
    # the grants array is sorted in a descending
    # order, so the highest grant is positioned
    # at index 0
    if surplus <= 0:
        return grantsArray[0]
     
    # start subtracting from surplus the
    # differences (“deltas”) between consecutive
    # grants until surplus is less or equal to zero.
    # Basically, we are testing out, in order, each
    # of the grants as potential lower bound for
    # the cap. Once we find the first value that
    # brings us below zero we break
    for i in range(0, n):
        surplus = surplus - ((i+1) * (arr[i] - arr[i+1]))  ##important
        if (surplus <= 0):
            break
  
    # since grantsArray[i+1] is a lower bound
    # to our cap, i.e. grantsArray[i+1] <= cap,
    # we  need to add to grantsArray[i+1] the
    # difference: (-total / float(i+1), so the
    # returned value equals exactly to cap.
    newCap = arr[i+1] + (-surplus / float(i+1))   ## important
    
    print(grantsArray)
    modified_array = [] 
    for i in range(0, len(grantsArray)):
        if (grantsArray[i] > int(newCap)):
            modified_array.append(newCap)
        elif (grantsArray[i] <= int(newCap)):
            modified_array.append(grantsArray[i])
            
    return newCap, modified_array

grantsArray = [2, 100, 50, 120, 1000]
newBudget = 190
print(find_grants_cap(grantsArray, newBudget))

In [ ]:
"""
========================= Bracket Match =============================
A string of brackets is considered correctly matched if every 
opening bracket in the string can be paired up with a later closing 
bracket, and vice versa. For instance, “(())()” is correctly matched, 
whereas “)(“ and “((” aren’t. For instance, “((” could become correctly 
matched by adding two closing brackets at the end, so you’d return 2.

Given a string that consists of brackets, write a function bracketMatch
that takes a bracket string as an input and returns the minimum number
of brackets you’d need to add to the input in order to make it correctly matched.

Explain the correctness of your code, and analyze its time and space complexities.

Examples:

input:  text = “(()”
output: 1

input:  text = “(())”
output: 0

input:  text = “())(”
output: 2
Constraints:

[time limit] 5000ms

[input] string text

1 ≤ text.length ≤ 5000
[output] integer
"""


"""Only parenthesis matcher -- Maruf"""
from collections import deque
def bracketMatcher(string):
    s = deque()
    balanced = True
    index = 0
    needed = 0
    while index < len(string):
        symbol = string[index]
        if symbol == "(":
            needed += 1
            s.append(symbol)
        else:
            if len(s)==0:
                needed += 1
                balanced = False
            else:
                needed -= 1
                s.pop()

        index = index + 1

    if balanced and len(s)==0:
        return [string, True, "Needed: {} (NO) parenthesis.".format(needed)]
    else:
        return [string, False, "Needed: {} more parenthesis.".format(needed)]

print("Only parenthesis matcher - Maruf")
print(bracketMatcher('((()))'))
print(bracketMatcher('(()()'))
print(bracketMatcher(')('))
print(bracketMatcher('))))))'))

"""Only parenthesis matcher -- Kiran"""
def bracket_match(text):
    q = deque() 
    if not text: return 0
    for c in text:
        if c == '(':
            q.append(c)
        else:
            # closing
            if not q: q.append(c)
            if q[-1] == '(': q.pop()
            else:   q.append(c)
    return len(q)
print("Only parenthesis matcher - Kiran")
print(bracket_match('((()))'))
print(bracket_match('(()()'))
print(bracket_match(')('))
print(bracket_match('))))))'))

"""All kinds of bracket matcher"""
def allBracketMatcher(string):
    s = deque()
    balanced = True
    index = 0
    needed = 0
    while index < len(string):
        symbol = string[index]
        if symbol in "([{":
            needed += 1
            s.append(symbol)
        else:
            if len(s)==0:
                needed += 1
                balanced = False
            else:
                needed -= 1
                s.pop()
        index += 1
        
    if balanced and len(s)==0:
        return [string, True, "Needed: {} (NO) bracket.".format(needed)]
    else:
        return [string, False, "Needed: {} more bracket(s).".format(needed)]

print("\nAll kinds of brackets matcher")
print(allBracketMatcher('{{([][])}()}'))
print(allBracketMatcher(')[{(()]'))
print(allBracketMatcher(']])[{{()]'))

In [ ]:
"""
Deletion Distance - Dynamic Programming
The deletion distance of two strings is the minimum number of characters you need to delete in the two 
strings in order to get the same string. For instance, the deletion distance between "heat" and "hit" is 3:

By deleting 'e' and 'a' in "heat", and 'i' in "hit", we get the string "ht" in both cases.
We cannot get the same string from both strings by deleting 2 letters or fewer.
Given the strings str1 and str2, write an efficient function deletionDistance that returns the deletion distance between them. Explain how your function works, and analyze its time and space complexities.

Time Complexity:  O(N⋅M). We have a nested loop that executes O(1) steps at every iteration, 
thus we the time complexity is O(N⋅M) where N and M are the lengths of str1 and str2, respectively.

Space Complexity:  O(N⋅M). We save every value of opt(i,j) in our memo 2D array, 
which takes O(N⋅M) space, where N and M are the lengths of str1 and str2, respectively.

Examples:

input:  str1 = "dog", str2 = "frog"
output: 3

input:  str1 = "some", str2 = "some"
output: 0

input:  str1 = "some", str2 = "thing"
output: 9

input:  str1 = "", str2 = ""
output: 0
"""
 
""" 
  Interviewee: 
  Ok so we can look through both strings, and delete
  chars that are not in both
  But the order matters
  so hit and heat.  If we delete e, a, i, we have th and ht (incorrect) -- TH and HT not the same
  Well what if we break down this to smaller problems
  So for heat and hit, we can solve hea and hi -- YES
  if we have h and h we can discard those, they don't impact deletion distance
  then we can look at he and hi, their deletion distance is 2 b/c e and i don't match
  then we can look at hea and hit, their deletion distance is 2 + 2 b/c a and i don't match
  ok ok, so h
  
  like this: (you're on the right track)
  
        ''         h        e                   a                  t
  ''    0          1        2                   3                  4
  h     1          0 (cc)   1(0+1for the e)     2(1+1 for a)       3 (2+1 for t)
  i     2          1        2(1+1)              3(2+1)             4(3+1) 
  t     3          2(1+1)   3                   4                  3(cc)
  
cc == cattycorner

"hit", "heat": Expected 3
[0, 1, 2, 3, 4]
[1, 0, 1, 2, 3]
[2, 1, 2, 3, 4]
[3, 2, 3, 4, 3]
"""
def deletionDistance(str1, str2):
    str1Len = len(str1)
    str2Len = len(str2)
    
    # allocate a 2D array with str1Len + 1 rows and str2Len + 1 columns
    memo = [[0 for i in range(str2Len+1)] for j in range(str1Len+1)]

    for i in range(0, str1Len+1):
        for j in range(0, str2Len+1):
            if (i == 0):
                memo[i][j] = j  # Rule #1: This is true because if one string is the empty string, 
                                # we have no choice but to delete all letters in the other string.
            elif (j == 0):
                memo[i][j] = i  # Same as Rule #1 
                
            elif (str1[i-1] == str2[j-1]):        # Rule #2: This holds since we don’t need to delete the last 
                memo[i][j] = memo[i-1][j-1]       # letters in order to get the same string, we simply 
                                                  # use the same deletions we would to the (i-1)'th and 
                                                  # (j-1)'th prefixes.
            else:
                # Rule #3: This holds since we need to delete at least one of the letters str1[i] or str2[j] 
                # and the deletion of one of the letters is counted as 1 deletion (hence the 1 in the 
                # formula). Then, since we’re left with either the (i-1)'th prefix of str1, or the
                # (j-1)'th prefix of str2, need to take the minimum between opt(i-1,j) and opt(i,j-1).
                # We, therefore, get the equation opt(i,j) = 1 + min(opt(i-1,j), opt(i,j-1)).
                memo[i][j] = 1 + min(memo[i-1][j], memo[i][j-1])

    for i in memo:
        print(i)
    return memo[str1Len][str2Len]

print("case 1: \"\", \"\" Expected 0")
print("result: ", deletionDistance("", ""))
print("case 2: \"\", hit Expected 3")
print("result: ", deletionDistance("", "hit"))
print("case 3: neat, \"\" Expected 4")
print("result: ", deletionDistance("neat", ""))
print("case 4: heat, hit Expected 3")
print("result: ", deletionDistance("heat", "hit"))
print("case 5: hot, not Expected 2")
print("result: ", deletionDistance("hot", "not"))
print("case 6: some, thing Expected 9")
print("result:", deletionDistance("some", "thing"))
print("case 7: abc, adbc Expected 1")
print("result: ", deletionDistance("abc", "adbc"))
print("case 8: awesome, awesome Expected 0")
print("result: ", deletionDistance("awesome", "awesome"))
print("case 9: ab, ba Expected 2")
print("result: ", deletionDistance("ab", "ba"))
print("case 10: hit, heat Expected 3")
print("result: ", deletionDistance("hit", "heat"))

In [ ]:
"""
MINIMUM NUMBER OF OPERATIONS TO TRANSFORM ONE STRING TO ANOTHER
# Dynamic Programming implementation to find minimum number of deletions and insertions
Source: https://www.geeksforgeeks.org/minimum-number-deletions-insertions-transform-one-string-another/
"""

# Returns length of length common subsequence for str1[0..m-1], str2[0..n-1]
def transform_with_min_ops(str1, str2, m, n):
    L = [[0 for i in range(n + 1)]for i in range(m + 1)]
    # Following steps build L[m+1][n+1] in bottom up fashion. 
    # Note that L[i][j] contains length of LCS of str1[0..i-1] and str2[0..j-1]
    for i in range(m + 1):
        for j in range(n + 1):
            if (i == 0 or j == 0):
                L[i][j] = 0
            elif (str1[i-1] == str2[j - 1]):
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j],L[i][j - 1])

    # L[m][n] contains length of LCS for X[0..n-1] and Y[0..m-1]
    return L[m][n]

def printMinDelAndInsert(str1, str2):
    """ Function to find minimum number of deletions and insertions """
    m = len(str1)
    n = len(str2)
    length = transform_with_min_ops(str1, str2, m, n)
    print("\nTo transform '" + str1 + "' to '" + str2 + "':")
    print("\tMinimum number of deletions from = ", m - length, sep = '')
    print("\tMinimum number of insertions from = ", n - length, sep = '')
    
# Driver Code
printMinDelAndInsert("heap", "pea")
printMinDelAndInsert("pea", "heap")
printMinDelAndInsert("", "")
printMinDelAndInsert("", "hit")
printMinDelAndInsert("neat", "")
printMinDelAndInsert("heat", "hit")
printMinDelAndInsert("hit", "heat")
printMinDelAndInsert("hot", "not")
printMinDelAndInsert("some", "thing")
printMinDelAndInsert("thing", "some")
printMinDelAndInsert("abc", "adbc")
printMinDelAndInsert("adbc", "abc")
printMinDelAndInsert("awesome", "awesome")
printMinDelAndInsert("ab", "ba")
printMinDelAndInsert("ba", "ab")

In [ ]:
"""
Merging 2 Packages
Given a package with a weight limit limit and an array arr of item weights, implement a function
get_indices_of_item_wights() that --
    1. Finds two items whose sum of weights equals the weight limit limit.
    2. Your function should return a pair [i, j] of the indices of the item weights, ordered such that i > j. 
    3. If such a pair doesn’t exist, return an empty array.
Analyze the time and space complexities of your solution.

Time Complexity: going over the array only once, performing constant time work for each weight and 
                assuming we have a good hash function with rare collisions, 
                we get a linear O(N) time complexity.
Space Complexity: O(N).
"""
def get_indices_of_item_wights(arr, limit):
    memo = {}
    for i in range(0, len(arr)):
        print(memo)
        if arr[i] in memo:
            return sorted([memo[arr[i]],i], reverse=True)
        else:
            memo[limit-arr[i]] =  i
    return []

def get_all_indices_and_value_pairs_of_item_wights(arr, limit):
    indices_pairs = []
    value_pairs = []
    memo = {}
    
    for i in range(0, len(arr)):
        if arr[i] in memo:
            indices_pairs.append(sorted([memo[arr[i]],i], reverse=True))         
            value_pairs.append(sorted([arr[memo[arr[i]]],arr[i]], reverse=True))
            ## if you want incremental/unsorted, simply remove the sorted() method from above:
        else:
            memo[limit-arr[i]] =  i
    for i in range(len(indices_pairs)):
        print("{a}:{b}\t== {c}".format(a=indices_pairs[i], b=value_pairs[i], c=limit))
        
arr, limit = [4, 15, 9, 18, 7, 16, 5, 23, 2, 10], 25
print("Input array:\t {k} ;\texpected:\t {a}".format(k=arr, a="[4, 3]"))
print("get_indices_of_item_wights:", get_indices_of_item_wights(arr, limit))
print("get_all_indices_and_value_pairs_of_item_wights: ")
get_all_indices_and_value_pairs_of_item_wights(arr, limit)
arr, limit = [9], 9  # expected []
result = get_indices_of_item_wights(arr, limit)
print("\nInput array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[]", b=result))
arr, limit = [4,4], 8  # expected [1,0]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[1,0]", b=result))
arr, limit = [4,4,1], 5  # expected [2,1]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[2,1]", b=result))
arr, limit = [4,6,10,15,16], 21  # expected [3,1]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[3,1]", b=result))
arr, limit = [4,6,10,15,16], 20   # expected [4,0]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[4,0]", b=result))
arr, limit = [12,6,7,14,19,3,0,25,40], 7 #expected [6,2]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[6,2]", b=result))


In [ ]:
"""alternate approach WIP"""
def deletion_distance(str1, str2):
    sub_string_dist = []
    for i in range(len(str1)):
        row = [0] * (len(str2))
        sub_string_dist.append(row)

    for i in range(len(str1)):
        for j in range(len(str2)):
            # if 0 row or 0 col, dist is from empty string
            if i == 0:
                sub_string_dist[i][j] = j  # argh == instead of =

            elif j == 0:
                sub_string_dist[i][j] = i
    # ok the above is good
    print(sub_string_dist)
    # my indexes are off by 1.
    # it's not accounting for the 'empty' str at the beginning

    for i in range(len(str1)):
        for j in range(len(str2)):
            # if equal, take cattycorner
            if str1[i] == str2[j]:
                sub_string_dist[i][j] = sub_string_dist[i - 1][j - 1]
            # if not, add one to min from adjacent
            elif str1[i] != str2[j]:
                sub_string_dist[i][j] = 1 + min(sub_string_dist[i - 1][j], sub_string_dist[i][j - 1])

    return sub_string_dist  # you should return the proper value from here, 
    # not the whole 2D array. once done, please run tests


def test():
    str1 = 'dog'
    str2 = 'frog'
    print(deletion_distance(str1, str2))


test() 

In [ ]:
""" Array of Array Products
Given an array of integers arr, you’re asked to calculate for each index
i the product of all integers except the integer at that index (i.e. except arr[i]).
Implement a function arrayOfArrayProducts that takes an array of integers and returns an array of the products.

Solve without using division and analyze your solution’s time and space complexities.

Examples:

input:  arr = [8, 10, 2]
output: [20, 16, 80] # by calculating: [10*2, 8*2, 8*10]

input:  arr = [2, 7, 3, 4]
output: [84, 24, 56, 42] # by calculating: [7*3*4, 2*3*4, 2*7*4, 2*7*3]
Constraints: 0 ≤ arr.length ≤ 20
"""
def calcProductArray_brute(arr): #brute force my version O(N^2), O(N)
    if len(arr) == 0 or len(arr) == 1:
        return arr
    result = []
    for i in range(len(arr)):
        product = 1
        if i == 0:
            for j in range(i+1, len(arr)):
                product *= arr[j]
        elif i == (len(arr)-1):
            for j in range(len(arr)-2, -1, -1):
                product *= arr[j]
        elif i > 0:
            totalBefore = arr[0]
            totalBeforeUpperBound = i - 1
            totalAfter = arr[i+1]
            totalAfterUpperBound = len(arr)-1
            for m in range(0, totalBeforeUpperBound):
                totalBefore *= arr[m+1]
            for n in range(i+1, totalAfterUpperBound):
                totalAfter *= arr[n+1]
            product = totalBefore * totalAfter
        result.append(product)

    return result

def calcProductArray_brute_pramp(arr): #brute force pramp version O(N^2), O(N)
    n = len(arr)
    if (n == 0 or n == 1):
        # nothing to multiply if n equals to 0 or 1
        return arr

    productArr = []
    for i in range(0, n):
        product = 1
        for j in range(0, n):
            if(i != j):
                product *= arr[j]

        productArr.append(product)

    return productArr

def calcProductArray(arr):   #O(N), O(N) 
    n = len(arr)
    if(n == 0 or n == 1):
        # no values to multiply if n equals to 0 or 1
        return arr

    productArr = []
    product = 1
    for i in range(0, n):
        productArr.append(product)
        product *= arr[i]
    
    product = 1
    for i in range(0, n):
        productArr[i] *= product
        product *= arr[i]

    return productArr

#Test Programs
def test():
    arr = [8, 10, 2]                     
    expected = [20, 16, 80]
    print("Expected:    \t", expected)
    print("Found(brute_maruf):\t", calcProductArray_brute(arr))
    print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr))
    print("Found(optimized):\t", calcProductArray(arr))

    arr2 = [2, 7, 3, 4]                  
    expected = [84, 24, 56, 42]
    print("Expected:    \t", expected)
    print("Found(brute_maruf):\t", calcProductArray_brute(arr2))
    print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr2))
    print("Found(optimized):\t", calcProductArray(arr2))

    arr3 = [-3,17,430,-6,5,-12,-11,5]    
    expected = [-144738000,25542000,1009800,-72369000,86842800,-36184500,-39474000,86842800]
    print("Expected:    \t", expected)
    print("Found(brute_maruf):\t", calcProductArray_brute(arr3))
    print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr3))
    print("Found(optimized):\t", calcProductArray(arr3))

    arr4 = []    
    expected = []
    print("Expected:    \t", expected)
    print("Found(brute_maruf):\t", calcProductArray_brute(arr4))
    print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr4))
    print("Found(optimized):\t", calcProductArray(arr4))

    arr5 = [5]    
    expected = []
    print("Expected:    \t", expected)
    print("Found(brute_maruf):\t", calcProductArray_brute(arr5))
    print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr5))
    print("Found(optimized):\t", calcProductArray(arr5))
    
    arr6 = [2,3,0,982,10]
    expected = [0,0,58920,0,0]
    print("Expected:    \t", expected)
    print("Found(brute_maruf):\t", calcProductArray_brute(arr6))
    print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr6))
    print("Found(optimized):\t", calcProductArray(arr6))

    arr7 = [2,2]
    expected = [2,2]
    print("Expected:    \t", expected)
    print("Found(brute_maruf):\t", calcProductArray_brute(arr7))
    print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr7))
    print("Found(optimized):\t", calcProductArray(arr7))
    
test()

In [ ]:
"""
Given a 2D array (matrix) inputMatrix of integers, create a function spiralCopy that copies
inputMatrix’s values into a 1D array in a spiral order, clockwise. Your function then should 
return that array. Analyze the time and space complexities of your solution.
See the illustration below to understand better what a clockwise spiral order looks like.

Pseudo: 
4 for loops
1 for loop to get top row, left to right
1 for loop to get left col, top to bottom
1 for loop to get bottom row, right to left
1 for loop to get right col, bottom to top
"""

def spiral_copy(inputMatrix):
    result = []
    if len(inputMatrix) == 0:
        return result
    elif len(inputMatrix) == 1:
        for col in range(0, len(inputMatrix[0])):
            result.append(inputMatrix[0][col])
        return result
    elif len(inputMatrix[0]) == 1:
        for row in range(0, len(inputMatrix)):
            result.append(inputMatrix[row][0])
        return result
    else:
        top_row = 0
        bottom_row = len(inputMatrix) - 1
        left_col = 0
        right_col = len(inputMatrix[top_row])-1
        while (top_row <= bottom_row) and (left_col <= right_col):  
            for col in range(left_col, right_col+1):         # move top left to top right
                result.append(inputMatrix[top_row][col])
            top_row += 1
            
            for row in range(top_row, bottom_row+1):         # move top right to down right
                result.append(inputMatrix[row][right_col])
            right_col = right_col - 1
            
            for col in range(right_col, left_col-1, -1):     # move down right to down left
                result.append(inputMatrix[bottom_row][col])
            bottom_row = bottom_row -1
            
            for row in range(bottom_row, top_row-1, -1):     # move down left to top left
                result.append(inputMatrix[row][left_col])
            left_col = left_col + 1
    return result      

print(spiral_copy([[1]]))     # expected [1]
print(spiral_copy([[1],[2]])) # expected [1,2]
print(spiral_copy([[1,2]]))   # expected [1,2]
print(spiral_copy([[1,2],[3,4]])) # expected [1,2,4,3]
print(spiral_copy([[1,2,3,4,5],[6,7,8,9,10]])) #expected [1,2,3,4,5,10,9,8,7,6]
print(spiral_copy([[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15],[16,17,18,19,20]]))
#expected [1,2,3,4,5,10,15,20,19,18,17,16,11,6,7,8,9,14,13,12]
  

In [22]:
""" Smallest Substring of All Characters -- KIRAN (recursive solution)
Given an array of unique characters arr and a string str, Implement a function 
getShortestUniqueSubstring that finds the smallest substring of str containing all the 
characters in arr. Return "" (empty string) if such a substring doesn’t exist.

Come up with an asymptotically optimal solution and analyze the time and space complexities.

Example:
input:  arr = ['x','y','z'], str = "xyyzyzyx"
output: "zyx"
"""

def get_shortest_unique_substring(arr, str):
    if not str:
        return ""
    if len(arr) == 1:
        if arr[0] == str:
            return str
        else:
            return ""
    res = [helper(arr, str[i:]) for i in range(len(str) - 1)]
    res = [x for x in res if x]

    result = ''
    max_len = len(str)
    #print(res)
    for elem in res:
        if len(elem) <= max_len:
            max_len = len(elem)
            result = elem
    return result

def helper(arr, string):
    from collections import defaultdict
    # print(arr, string)
    count = len(arr)
    seen = defaultdict(int)
    subString = []
    for c, i in enumerate(string):
        if i in arr:
            seen[i] += 1
            # print(seen)
            if len(seen) == count:
                # print(seen, string[:c+1])
                return string[:c + 1]

    return ''

print("Result:", get_shortest_unique_substring(["A"], "" )) # Expected: ""
print("Result:", get_shortest_unique_substring(["A"], "B" )) # Expected: ""
print("Result:", get_shortest_unique_substring(["A"], "A" )) # Expected: "A"
print("Result:", get_shortest_unique_substring(["A","B","C"], "ADOBECODEBANCDDD" )) # Expected: "BANC"
print("Result:", get_shortest_unique_substring(["A","B","C","E","K","I"], "KADOBECODEBANCDDDEI" )) # Expected: "KADOBECODEBANCDDDEI"
print("Result:", get_shortest_unique_substring(["x","y","z"], "xyyzyzyx" )) # Expected: "zyx"
print("Result:", get_shortest_unique_substring(["x","y","z","r"], "xyyzyzyx" )) # Expected: ""

Result: 
Result: 
Result: A
Result: BANC
Result: KADOBECODEBANCDDDEI
Result: zyx
Result: 


In [ ]:
"""Note"""
s = 'ABCDEFGH'
res= [s[i:] for i in range(len(s)-1)]
print(res)
res = [x for x in res if x]
print(res)

In [17]:
"""
Island Count
Given a 2D array binaryMatrix of 0s and 1s, implement a function getNumberOfIslands that returns the number of islands of 1s in binaryMatrix.

An island is defined as a group of adjacent values that are all 1s. A cell in binaryMatrix is considered adjacent to another cell if they are next to each either on the same row or column. Note that two values of 1 are not part of the same island if they’re sharing only a mutual “corner” (i.e. they are diagonally neighbors).

Explain and code the most efficient solution possible and analyze its time and space complexities.

Example:

input:  binaryMatrix = [ [0,    1,    0,    1,    0],
                         [0,    0,    1,    1,    1],
                         [1,    0,    0,    1,    0],
                         [0,    1,    1,    0,    0],
                         [1,    0,    1,    0,    1] ]

output: 6 

Solution:
To solve this problem, we’ll traverse binaryMatrix and every time we come across a cell of 1 
we’ll do the following: Change that cell and all its vertically and horizontally (but not diagonally)
adjacent 1s into -1s. We do this “expansion” in order to avoid recounting of islands. 
Increment islands - which is our counter for number of islands - by 1. Expanding from a cell whose 
value is 1 to other adjacent 1s in binaryMatrix is similar to running a Breadth-First Search (BFS) 
or a Depth-First Search (DFS).
In our case, we’ll avoid using a recursion and instead opt for an iterative approach to expand to
all adjacent 1s. We do so by using queue that holds the next indices to visit. We keep expanding to
other adjacent 1s as long as the queue is not empty. Whenever we encounter a value of -1 in our traversal,
we ignore it since it is part on an island we’ve already counted.

Time Complexity: let N and M be the numbers of columns and rows in binaryMatrix, respectively. 
Each cell in binaryMatrix is visited a constant number of times. Once during the iteration and 
up to 4 times during an island expansion. Therefore, the time complexity is linear in the size 
of the input, i.e. O(N⋅M).
Space Complexity: since we are allocating a queue in the algorithm, the space complexity is 
linear O(N⋅M). For instance, consider a matrix that is all 1s.
"""
def getNumberOfIslands(binaryMatrix):
    print("\ninput matrix: ") #printing the input
    for row in binaryMatrix:
        print(row)
        
    islands = 0
    rows = len(binaryMatrix)    # number of rows
    cols = len(binaryMatrix[0]) # number of columns
    for i in range(0, rows):
        for j in range(0, cols):
            if (binaryMatrix[i][j] == 1):
                markIsland(binaryMatrix, rows, cols, i, j)
                islands += 1 
    print("number of islands: ", islands)
    return islands

def markIsland(binaryMatrix, rows, cols, i, j):
    from collections import deque
    q = deque()   #queue
    q.append([i,j])
    while len(q)>0:
        item = q.popleft()
        x = item[0]
        y = item[1]
        if (binaryMatrix[x][y] == 1):
            binaryMatrix[x][y] = -1
            pushIfValid(q, rows, cols, x-1, y)
            pushIfValid(q, rows, cols, x, y-1)
            pushIfValid(q, rows, cols, x+1, y)
            pushIfValid(q, rows, cols, x, y+1)

def pushIfValid(q, rows, cols, x, y):
    if (x >= 0 and x < rows and y >= 0 and y < cols):
        q.append([x,y])      

getNumberOfIslands([[0]]) # Expected: 0
getNumberOfIslands([[1]]) # Expected: 1
getNumberOfIslands([[1,0,1,0]]) # Expected: 2
getNumberOfIslands([[1,0,1,0],[0,1,1,1],[0,0,1,0]]) # Expected: 2
getNumberOfIslands([[1,0,1,0],[0,1,1,1],[0,0,1,0],[1,1,0,0],[0,1,0,1]]) # Expected: 4
getNumberOfIslands([[0,1,0,1,0],[0,0,1,1,1],[1,0,0,1,0],[0,1,1,0,0],[1,0,1,0,1]]) # Expected: 6
getNumberOfIslands([[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]]) # Expected: 1


input matrix: 
[0]
number of islands:  0

input matrix: 
[1]
number of islands:  1

input matrix: 
[1, 0, 1, 0]
number of islands:  2

input matrix: 
[1, 0, 1, 0]
[0, 1, 1, 1]
[0, 0, 1, 0]
number of islands:  2

input matrix: 
[1, 0, 1, 0]
[0, 1, 1, 1]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 1, 0, 1]
number of islands:  4

input matrix: 
[0, 1, 0, 1, 0]
[0, 0, 1, 1, 1]
[1, 0, 0, 1, 0]
[0, 1, 1, 0, 0]
[1, 0, 1, 0, 1]
number of islands:  6

input matrix: 
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
number of islands:  1


1

In [31]:
"""Sales Path
The car manufacturer Honda holds their distribution system in the form of a tree (not necessarily binary).
The root is the company itself, and every node in the tree represents a car distributor that receives cars 
from the parent node and ships them to its children nodes. The leaf nodes are car dealerships that sell 
cars direct to consumers. In addition, every node holds an integer that is the cost of shipping a car to it.
A path from Honda’s factory to a car dealership, which is a path from the root to a leaf in the tree, is 
called a Sales Path. The cost of a Sales Path is the sum of the costs for every node in the path. 
For example, in the tree above one Sales Path is 0→3→0→10, and its cost is 13 (0+3+0+10).
Honda wishes to find the minimal Sales Path cost in its distribution tree. Given a node rootNode, 
write a function getCheapestCost that calculates the minimal Sales Path cost in the tree.
Take for example the tree below:
         0
     /   |   \
   5     3     6
  /     / \   / \
 4     2   0  1  5
      /   /
      1  10
       \
        1
Implement your function in the most efficient manner and analyze its time and space complexities.
For example:
Given the rootNode of the tree in diagram above
Your function would return: 7 since it’s the minimal Sales Path cost (there are actually two 
Sales Paths in the tree whose cost is 7: 0→6→1 and 0→3→2→1→1)

Obviously iterating through all paths again and again is not a good solution, since its wasteful
in terms of time and memory. But intuitively if we find a solution that uses previous calculations 
somehow. This hints that the solution should involve recursion in some manner.
First we notice that if the root is also a leaf, the best Sales Path, is simply the value in the 
node itself. This is the base case for the solution. If the root has children, then the minimal 
Sales Path is also a minimal path from the root’s child. Thus, if we already know the minimal cost
for the root’s children, then the minimal cost for the root is simply the minimum of the values for
its children plus the value stored in the root itself.

Time Complexity:
let N be the number of nodes in the tree. Notice that getCheapestCost is applied
to every node exactly once. Therefore, there are overall O(N) calls to getCheapestCost.
Space Complexity: 
every time the function recurses, it consumes only a constant amount of space. 
However, due to the nature of the recursion we used, the stack call holds N instances of 
getCheapestCost which makes the total space complexity to be O(N).
"""

# A node 
class Node:
  # Constructor to create a new node
  def __init__(self, cost):
    self.cost = cost
    self.children = []
    self.parent = None
    
def getCheapestCost(rootNode):
    import sys
    n = len(rootNode.children)
    if (n == 0): 
        return rootNode.cost
    else:
        # initialize minCost to the largest integer in the system
        minCost = sys.maxsize
        for i in range(0, n):
            tempCost = getCheapestCost(rootNode.children[i])
            if (tempCost < minCost):
                minCost = tempCost
    return minCost + rootNode.cost

"""=========TESTING==========
         0
     /   |   \
   5     3     6
  /     / \   / \
 4     2   0  1  5
      /   /
      1  10
       \
        1
"""
node0 = Node(0) 
node0.children = [Node(10)]
node1 = Node(1) 
node1.children = [Node(1)]
node2 = Node(2) 
node2.children = [node1]
node5 = Node(5) 
node5.children = [Node(4)]
node3 = Node(3) 
node3.children = [node2, node0]
node6 = Node(6) 
node6.children = [Node(1),Node(5)]
root = Node(0)
root.children = [node5, node3, node6]
print(getCheapestCost(root)) #expected 7

7
